Here I import the necessary packages

In [1]:
import numpy as np
import keras
import cv2
import pandas as pd
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import os
%matplotlib inline

Using TensorFlow backend.


Here I initialise the file paths for the images and the labels

In [2]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
labels_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv'

Here I import the images and labels into a training data frame using pandas

In [3]:
dft = pd.read_csv(labels_path, skiprows=lambda x: x in range(1,5000,5))
dft["smiling"] = dft["smiling"].replace(to_replace=[-1], value=['Frown'])
dft["smiling"] = dft["smiling"].replace(to_replace=[1], value=['Smile'])
dft["gender"] = dft["gender"].replace(to_replace=[-1], value=['Female'])
dft["gender"] = dft["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dft["smiling"])
one_hot_g = pd.get_dummies(dft["gender"])
dft = dft.drop(columns=["gender", "smiling"])
dft = dft.join(one_hot_g)
dft = dft.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(
    dataframe=dft, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Frown", "Smile"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 4000 validated image filenames.


Here I do the same thing but for a testing data frame

In [4]:
dfv = pd.read_csv(labels_path, skiprows=lambda x: x not in range(0,5000,5))
dfv["smiling"] = dfv["smiling"].replace(to_replace=[-1], value=['Frown'])
dfv["smiling"] = dfv["smiling"].replace(to_replace=[1], value=['Smile'])
dfv["gender"] = dfv["gender"].replace(to_replace=[-1], value=['Female'])
dfv["gender"] = dfv["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dfv["smiling"])
one_hot_g = pd.get_dummies(dfv["gender"])
dfv = dfv.drop(columns=["gender", "smiling"])
dfv = dfv.join(one_hot_g)
dfv = dfv.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(
    dataframe=dfv, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Frown", "Smile"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 999 validated image filenames.


The above code generates batches of data to feed into the neural network architecture below

In [5]:
model = Sequential()
model.add(InputLayer(input_shape=(55,45,1)))
model.add(Conv2D(filters=100, kernel_size=(3,3), strides=2, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=200, kernel_size=(2,2), strides=2, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=400,kernel_size=(2,2),activation="relu"))
model.add(Flatten())
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

The following code just compiles the neural network model

In [6]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
train_steps = train_generator.n//train_generator.batch_size
test_steps = test_generator.n//test_generator.batch_size

The following code just trains the model

In [9]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=train_steps,
                    validation_data=test_generator,
                    validation_steps=test_steps,
                    epochs=25)

Epoch 1/25
40/40 [==============================] - 20s 502ms/step - loss: 0.7734 - accuracy: 0.6185 - val_loss: 0.6877 - val_accuracy: 0.5778
Epoch 2/25
40/40 [==============================] - 22s 542ms/step - loss: 0.5186 - accuracy: 0.7513 - val_loss: 0.6864 - val_accuracy: 0.5239
Epoch 3/25
40/40 [==============================] - 20s 510ms/step - loss: 0.4412 - accuracy: 0.7897 - val_loss: 0.7027 - val_accuracy: 0.5017
Epoch 4/25
40/40 [==============================] - 19s 487ms/step - loss: 0.3723 - accuracy: 0.8303 - val_loss: 0.7791 - val_accuracy: 0.5195
Epoch 5/25
40/40 [==============================] - 20s 493ms/step - loss: 0.3276 - accuracy: 0.8570 - val_loss: 0.8124 - val_accuracy: 0.5139
Epoch 6/25
40/40 [==============================] - 19s 483ms/step - loss: 0.2687 - accuracy: 0.8905 - val_loss: 0.7633 - val_accuracy: 0.5161
Epoch 7/25
40/40 [==============================] - 20s 488ms/step - loss: 0.2261 - accuracy: 0.9053 - val_loss: 1.0662 - val_accuracy: 0.5095